In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt

In [3]:
pwd()

u'/home/brianna'

In [4]:
projects = pd.read_csv('Documents/wise/Data/opendata_projects.csv')

In [4]:
projects.shape

(771929, 44)

Function to count the number of days between start_date and complete_date

In [5]:
from dateutil.parser import parse
import math
def dayCount(date0, date1):
    if isinstance(date0, float) or isinstance(date1, float):
        return 'NA'
    try:
        delta = parse(date0) - parse(date1)
    except Exception, e:
        print date1, type(date1), date0, type(date0)
        raise e
    return delta

Apply dayCount function over dataframe

In [6]:
def dateCalc(row):
    return dayCount(row['date_completed'], row['date_posted'])
dateDiff = projects.apply(dateCalc, 1)

In [7]:
dtc = pd.DataFrame({'days_to_completion': pd.Series(dateDiff)})

Add the dataframe containing dayCounts to projects dataframe

In [26]:
total = pd.concat([projects, dtc], axis = 1)

Narrow down raw data to include only variables of interest

In [9]:
varsOfInt = total.loc[:,['school_state','funding_status','poverty_level',
                           'primary_focus_area','primary_focus_subject',
                            'grade_level','total_price_excluding_optional_support',
                            'date_posted','date_completed','days_to_completion']]

Subset to include only funded projects

In [10]:
fundedProjects = varsOfInt[varsOfInt.funding_status == 'completed']

Get column of school state for all projects, and for funded projects

In [13]:
states = total['school_state']
states_funded = fundedProjects[['school_state','days_to_completion']]

Get project funding percent by state

In [14]:
total_states_count = states.value_counts()
funded_states_count = states_funded['school_state'].value_counts()
funding_pct_by_state = funded_states_count/total_states_count

Get average days to completion for each state

In [15]:
st = pd.DataFrame({'counts': pd.Series(total_states_count)})
fps = pd.DataFrame({'funding_percent': pd.Series(funding_pct_by_state)})
st_counts_pct = pd.concat([st, fps], axis = 1)

Write st_counts_pct, fundedProjecst to csv

In [16]:
st_counts_pct.to_csv("states.csv")
fundedProjects.to_csv("funded.csv")

Subset projects to include only open projects

In [17]:
openProjects = varsOfInt[varsOfInt.funding_status == 'live']

Add column of current date so we can count open days

In [18]:
openProjects['current_date'] = '2014-11-10'

/home/brianna/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
openProjects['current_date'].value_counts(sort = True)

2014-11-10    41871
dtype: int64

Apply dayCount over open projects

In [20]:
def dateCalc2(row):
    return dayCount(row['current_date'], row['date_posted'])
dateDiff2 = openProjects.apply(dateCalc2, 1)

In [21]:
dtc2 = pd.DataFrame({'days_open': pd.Series(dateDiff2)})

Add the dataframe containing daysOpen to projects dataframe

In [38]:
total2 = pd.concat([total, dtc2], axis = 1)

Write data file including days open and days to funding to csv

In [44]:
#total2[['funding_status','date_posted','date_completed','days_open','days_to_completion']]
#total2[total2.funding_status == 'live'].head
total2.to_csv("projects_modified.csv")